# SQL in Python - Connecting to and retrieving data from PostgreSQL

Previously, you have learned how to connect to a SQL database by using a SQL client such as DBeaver.
Apart from connecting to databases, DBeaver also allows you to run SQL queries against the database, create new tables and populate them with data as well as retrieving the data.

Populating tables with data that you have locally on your machine usually requires you to save it in a file, like a CSV, and import it using the DBeaver UI.

Often, before you reached the final step of uploading your dataset, you have performed data cleaning procedures to bring your data into shape. This means we would import the data into Python, clean it, export it to a CSV file, import it into DBeaver and upload the data into the database.

This process requires multiple steps and more than one software. Fortunately, we can reduce the steps by connecting to the database from Python directly, eliminating the need for a separate SQL client.

**In this notebook you will see 2 ways to connect to SQL-Databases and export the data to a CSV file.**


## Creating a connection to a PostgreSQL database with Python

There are 2 python packages that are the "go-to" when it comes to connecting to SQL-Databases: `psycopg2` and `sqlalchemy` 

First an example with psycopg2:

In [1]:
import pandas as pd # to read sql data into a pandas dataframe
import psycopg2 # to connect to SQL database


In order to create a connection to our PostgreSQL database we need the following information (usually posted by coaches during SQL days in Slack):

- database = name of the database
- user = name of the user
- password = password of the user
- host = address of the machine the database is hosted on
- port = virtual gate number through which communication will be allowed

Because we don't want that the database information is published on GitHub, we put it into a `.env` file which is added into the `.gitignore`. 

In these kind of files you can store information that is not supposed to be published.
With the `dotenv` package you can read the `.env` files and get the variables in there.
The file was 'force added' to the repo using `git add -f .env` command. Please follow instructions inside the `.env` file to ensure you have the right credentials inside. 

Then, run the following code cell (no need to edit): 

In [2]:
import os # provides functions for interacting with operating system
from dotenv import load_dotenv # reads key-value pairs from a .env file and can set them as environment variables

load_dotenv() # takes environment variables from .env

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

The function from the psycopg2 package to create a connection is called `connect()`. 

`connect()` expects the parameters listed above as input in order to connect to the database. 

>**Note**: If you edited your `.env` file correcty, but still get an error when trying to connect, "Restart" your Kernel. 

In [3]:
# Create / open connection object conn (no need to edit code)
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

## Retrieving data from the database

Before we can use our connection to get data, we have to create a cursor. A cursor allows Python code to execute PostgreSQL commmands in a database session.
A cursor has to be created with the `cursor()` method of our connection object conn.

In [ ]:
cur = conn.cursor() # create cursor for our opened connection in object conn

Now we can run SQL-Queries with `cur.execute('QUERY')` and then run `cur.fetchall()` to get the data:

In [ ]:
cur.execute('SELECT * FROM datasets.kaggle_survey LIMIT 10') # executes given SQL query
cur.fetchall() # gets data called by query

With `conn.close()` you can close the connection again.

In [ ]:
#close the connection
conn.close()

But we want to work with the data. The easiest way is to import the data into pandas dataframes. We can use `pd.read_sql_query` or `pd.read_sql_table` or for convenience `pd.read_sql`.

This function is a convenience wrapper around `read_sql_table` and `read_sql_query` (for backward compatibility). It will delegate to the specific function depending on the provided input. A SQL query will be routed to `read_sql_query` , while a database table name will be routed to `read_sql_table` . Note that the delegated function might have more specific notes about their functionality which are not listed here. Find more in the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html).

In [ ]:
# Open connection again because we closed it (no need to edit code)
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

In [ ]:
# import the data into a pandas dataframe
query_string = "SELECT * FROM datasets.kaggle_survey LIMIT 10" # define SQL query
df_psycopg = pd.read_sql(query_string, conn) # read queried data from SQL database into pandas dataframe

In [ ]:
#close the connection
conn.close()

In [ ]:
df_psycopg.head() # look at first five lines of dataframe

#### SQLALCHEMY

`sqlalchemy` works similarly. Here you have to create an engine with the database string (a link that includes every information we entered in the conn object)

In [ ]:
from sqlalchemy import create_engine # for creating an engine

#read the database string DB_STRING from the .env
load_dotenv()

DB_STRING = os.getenv('DB_STRING') # gets database string DB_STRING from .env file and assigns it as value for new variable DB_STRING

db = create_engine(DB_STRING) # creates engine from database string DB_STRING

And then you can import that engine with a query into a pandas dataframe.

In [ ]:
import pandas as pd # if not done already

In [ ]:
#import the data to a pandas dataframe
query_string = "SELECT * FROM datasets.kaggle_survey" # write SQL-query into variable query_string
df_sqlalchemy = pd.read_sql(query_string, db) # read queried data from SQL database into pandas dataframe

In [ ]:
df_sqlalchemy.head() # look at first five lines of dataframe

Because we don't want to run the queries over and over again we can then export the data into a csv and import that file into our main jupyter notebook: [Visualisation_Exercise](4_Visualisation_Exercise.ipynb)

In [ ]:
#export the data to a csv-file
df_sqlalchemy.to_csv('data/kaggle_survey.csv',index=False)